In [9]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
def load_images_and_masks(images_path, masks_path, image_size):
    images = []
    masks = []
    
    for filename in os.listdir(images_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image = cv2.imread(os.path.join(images_path, filename))
            image = cv2.resize(image, (image_size, image_size))
            images.append(image)
            
            mask = cv2.imread(os.path.join(masks_path, filename), cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (image_size, image_size))
            masks.append(mask)
    
    images = np.array(images) / 255.0
    masks = np.array(masks) / 255.0
    masks = np.expand_dims(masks, axis=-1)
    
    return images, masks

images, masks = load_images_and_masks('F:\\super_resolution_project\\hr_crowd_img', 'F:\\super_resolution_project\\masks_img', 128)

In [12]:
def unet_model(input_size=(128, 128, 3)):
    inputs = tf.keras.layers.Input(input_size)
    
    # Encoder
    c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    
    c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    
    c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    
    c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)
    
    # Bottleneck
    c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p4)
    c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c5)
    
    # Decoder
    u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u6)
    c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)
    
    u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u7)
    c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)
    
    u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u8)
    c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)
    
    u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u9)
    c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c9)
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = unet_model()

In [13]:
# Early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('unet_best_model.keras', save_best_only=True, monitor='val_loss')

# Split the data into training and validation sets
split_idx = int(len(images) * 0.8)
train_images, val_images = images[:split_idx], images[split_idx:]
train_masks, val_masks = masks[:split_idx], masks[split_idx:]

# Train the model
model.fit(train_images, train_masks, validation_data=(val_images, val_masks), batch_size=8, epochs=20, callbacks=[early_stopping, model_checkpoint])

Epoch 1/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 141s 907ms/step - accuracy: 0.9716 - loss: 0.1842 - val_accuracy: 0.9619 - val_loss: 0.1208
Epoch 2/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 141s 947ms/step - accuracy: 0.9746 - loss: 0.0844 - val_accuracy: 0.9619 - val_loss: 0.1067
Epoch 3/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 141s 948ms/step - accuracy: 0.9734 - loss: 0.0813 - val_accuracy: 0.9619 - val_loss: 0.1038
Epoch 4/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 141s 948ms/step - accuracy: 0.9752 - loss: 0.0734 - val_accuracy: 0.9619 - val_loss: 0.1097
Epoch 5/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 142s 953ms/step - accuracy: 0.9726 - loss: 0.0756 - val_accuracy: 0.9619 - val_loss: 0.0972
Epoch 6/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 143s 960ms/step - accuracy: 0.9743 - loss: 0.0692 - val_accuracy: 0.9621 - val_loss: 0.0931
Epoch 7/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 141s 951ms/step - accuracy: 0.9732 - loss: 0.0704 - val_accuracy: 0.9619 - val_loss: 0.0981
Epoch 8/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 143s 959ms/step - accuracy: 0.9738 -

In [19]:
def predict_and_save_rois(model, input_folder, output_folder, image_size=128, roi_size=(128, 128)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            test_image = cv2.imread(image_path)
            original_size = test_image.shape[:2]  # Save original size for later
            test_image_resized = cv2.resize(test_image, (image_size, image_size)) / 255.0
            test_image_resized = np.expand_dims(test_image_resized, axis=0)
            
            # Predict the mask
            predicted_mask = model.predict(test_image_resized)[0]
            predicted_mask = (predicted_mask > 0.5).astype(np.uint8)
            
            # Resize the predicted mask to the original image size
            predicted_mask_resized = cv2.resize(predicted_mask, (original_size[1], original_size[0]))
            
            # Find contours in the mask
            contours, _ = cv2.findContours(predicted_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Save each detected ROI (face) to the output folder
            for i, contour in enumerate(contours):
                x, y, w, h = cv2.boundingRect(contour)
                roi = test_image[y:y+h, x:x+w]
                # Resize the ROI
                roi_resized = cv2.resize(roi, roi_size)
                roi_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_roi_{i}.png")
                cv2.imwrite(roi_path, roi_resized)
                print(f"Saved ROI to: {roi_path}")

# Use the function to predict and save ROIs from all images in the folder
input_folder = 'F:\\super_resolution_project\\hr_crowd_img'
output_folder = 'F:\\super_resolution_project\\cropped_faces'
predict_and_save_rois(model, input_folder, output_folder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1002bf000c3801e66_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1002bf000c3801e66_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1002bf000c3801e66_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1002bf000c3801e66_roi_3.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1005250002ecf8aae_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1005250002ecf8aae_roi_1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,10313900009b59dd5_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Saved ROI to: F:\super_resolution_project\cropped_f

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1500e000314f6567_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1500e000314f6567_roi_1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,15155000b08bbd78_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,15155000b08bbd78_roi_1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1517c000b0fd7106_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1537e00042ab67c2_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1537e00042ab67c2_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,1537e00042ab67c2_roi_2.png
Saved ROI to: F:\super_resolution_project\

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,24284000e2dd1a91_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2434f000d100b269_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_roi_3.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_roi_4.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_roi_5.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2456300002aac469_

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_3.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_4.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_5.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,2842a00041949dc2_roi_6.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Saved ROI to: F:\supe

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,45156000d3b261ad_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,4517b0000d8d575f_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,4517b0000d8d575f_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,4517b0000d8d575f_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1066412,4517b0000d8d575f_roi_3.png
Saved ROI to:

...

Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_22.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_23.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_24.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_25.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_26.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_27.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_28.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_29.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_30.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a_roi_31.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,38a7d000b27ac83a

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4ac9a000bb3f6094_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4ac9a000bb3f6094_roi_1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4af210004ed6bae7_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4af210004ed6bae7_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4af210004ed6bae7_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4af210004ed6bae7_roi_3.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4cf530002025886b_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,4cf530002025886b_roi_1.png
Saved ROI to: F:\super_resolution_project\

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6b93b000bcd7394c_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6bb9f00077d0febf_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6bb9f00077d0febf_roi_1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6c5ab000be6b59de_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6c5ab000be6b59de_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6c5ab000be6b59de_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6c5ab000be6b59de_roi_3.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6c5ab000be6b59de_roi_4.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,6c5ab000be6b59de_roi_5.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74ae9000c7f56d40_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74ae9000c7f56d40_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74ae9000c7f56d40_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74ae9000c7f56d40_roi_3.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74b70000a30ebb63_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74b70000a30ebb63_roi_1.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74b70000a30ebb63_roi_2.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,74b70000a30ebb63_roi_3.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,75b34000cb383e3b_

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,8f80b000a8de8c9c_roi_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,8fec900001b5bc40_roi_0.png
Saved ROI to: F:\super_resolution_project\cropped_faces\1074471,8fec900001b5bc40_roi_1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Sa